# Finite differences in 1D

We approximate the boundary value problem consisting of the following partial differential equation (PDE), actually an ODE in this case:
$$
-u''(x) + u(x) = f(x), \qquad  0 < x < 1.
$$
This equation is supplemented with  Dirichlet boundary conditions 
$$
u(0) = u(1) = 0
$$
at both ends. 

We quickly illustrate the standard finite difference method for solving this, taking the opportunity to also ensure that we are all on the same page regarding python and numpy prerequisites. Let's begin by importing `numpy`.

In [ ]:
import numpy as np

## Making the finite difference system


Finite differences approximate the derivative of a real-valued 
function  $f$ (of  a single real variable $x$)
by 

$$
f'(x) \approx {f(x+h/2) - f(x-h/2) \over h}
$$

We take one further step and approximate the second derivative by 
$$
\begin{aligned}
f''(x) 
& 
\approx { f'(x+h/2) - f'(x-h/2) \over h }
\\
& \approx { \left(\frac{f(x+h/2+h/2) - f(x+h/2-h/2)}{h}\right) - \left(\frac{f(x-h/2) - f(x-h/2-h/2)}{h} \right)\over h }
\\
& \approx {f(x+h) - 2f(x) + f(x-h)\over h^2}
\end{aligned}
$$
This is the **Central Difference Formula** for the second derivative.

The **matrix** of the *central second finite difference* operator on a grid of *just* 5 equally spaced points, two of which have zero boundary conditions, can be "made by hand" as a `numpy` array, as follows, postponing the multiplication by the factor $1/h^2$.

In [ ]:
A = np.array([[ 2, -1,  0], 
              [-1,  2, -1], 
              [ 0, -1,  2]])
A

For large number of grid points, we need an automatic way to make this matrix. Numpy provides many ways to create matrices quickly. For example, the `diag` command generates matrices with input entries in the diagonal, superdiagonals, or subdiagonals.

In [ ]:
N = 10
2 * np.diag(np.ones(N))

In [ ]:
A = 2 * np.diag(np.ones(N)) +    \
    np.diag(-np.ones(N-1), -1) + \
    np.diag(-np.ones(N-1), 1)  + \
    np.eye(N)
A

## Solving the difference equation system

We solve the finite difference equations using the built-in inverse routine in numpy's `linalg` submodule. Note how we create functions using `def` in python and how we use `@` for numpy matrix multiply.

In [ ]:
def solve(f, h):
    size = len(f)
    A = (1/h**2) * (2 * np.diag(np.ones(size))    + \
                    np.diag(-np.ones(size-1), -1) + 
                    np.diag(-np.ones(size-1), 1)) + np.eye(size)
    return np.linalg.inv(A) @ f

Now you have an approximate solution to the boundary value problem in the returned vector. Is this any good? 

## Check for correctness

**Verification** is a key step while studying numerical methods.

We will verify that the `solve` function is solving as expected by the **method of manufactured solutions**, which is just a fancy name for  the following simple idea: pick your favorite smooth function $u$ satisfying the boundary conditions, then generate the right hand side $f$ that would give your $u$ as the exact solution by applying the differential operator to $u$. 

In the check below, I will use
$$
u = \sin(x)
$$
on an interval $(0, 3\pi)$ so that the zero boundary conditions hold. Then put $f = -u'' + u = 2\sin(x)$. Let's provide this $f$ to `solve` and see whether it outputs an approximation to $u = \sin(x)$.

An equally spaced grid in an interval is easily made using the `linspace` facility.

In [ ]:
np.linspace(0, 3*np.pi, num=10)

Let's "manufacture"  the data $f$ for the manufactured solution $u=\sin(x)$.

In [ ]:
N = 30    # the system becomes more expensive to solve for large N
h = 3*np.pi / N
x = np.linspace(0, 3*np.pi, num=N)
f = 2 * np.sin(x)

One point to note about the boundary points: when we solve, we make sure not to give the end point values in `f` (as the solution there is already determined by the boundary conditions).  Restricting $f$ outside of these points is done by **slicing** in numpy (which you should definitely learn if you don't know already) as in `f[1:-1]`.

In [ ]:
u = solve(f[1:-1], h)

For visualizing the solution, we use `matplotlib` module.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(u, '.-')

How can you make sure the end points with the known zero solution values  are also included in the final plot? This is again done using slicing. 

In [ ]:
uu = np.zeros(N)
uu[1:-1] = u

Let's conclude by comparing the exact solution with the numerical solution by plotting both in the same scene. 

In [ ]:
plt.plot(uu, '.', label='computed solution')
plt.plot(np.sin(x), ':', label='exact solution', alpha=0.5)
plt.grid(True)
plt.legend();

Clearly, the `solve` function captured something close to the exact solution.

Going back through this notebook, you can easily change the mesh parameter to make $h$ smaller. Then repeating the above steps, you will see that the discrete solution points gets closer to exact solution curve. When we study finite elements, we will think of the discrete solution also as a function, not as a set of discrete values as above. We will then measure the difference of exact and discrete solutions using function space norms like the $L^2$ norm.